In [1]:
load('Imports/Utility.sage', 'Imports/Point.sage', 'Imports/Line.sage', 'Imports/TritangentPlane.sage', 'Imports/Group.sage', 'Imports/Cubic.sage')
import multiprocessing as mp
import pickle

In [2]:
xx = QQ['xx'].0

In [3]:
Q.<alpha> = NumberField(xx^2-5)

In [4]:
P.<x,y,z,t,b,c,d,e,f,l,m> = PolynomialRing(Q)
P

Multivariate Polynomial Ring in x, y, z, t, b, c, d, e, f, l, m over Number Field in alpha with defining polynomial xx^2 - 5

In [5]:
cubic_new = e*f*(2*x^2*y-2*x*y^2+x*z^2-x*z*t-y*t^2+y*z*t)+b*c*(x-t)*(x*z+y*t)+c*c*(z+t)*(y*t-x*z)+d*c*(y-z)*(x*z+y*t)+(e+f)*c*(x-y)*(y*t-x*z)

In [6]:
sing_cubics = (-1) * (-c + f) * (-c + e) * c * (c + f) * (c + e) * (-e + f)^2 * (-c*d + c*f + e*f) * (-c*d + c*e + e*f) * (-c^2 - c*d + e*f) * (b*c - c*f + e*f) * (b*c - c*e + e*f) * (b*c - c*d + 2*e*f) * (b*c - c^2 + e*f) * (b*c^2 + c^2*d + b*c*f - 2*c^2*f - c*d*f + 2*e*f^2) * (b*c^2 + c^2*d + b*c*e - 2*c^2*e - c*d*e + 2*e^2*f) * (-b*c^3 - 2*b*c^2*d + c^3*d + b*c^2*e + c^2*d*e + b*c^2*f + c^2*d*f + 3*b*c*e*f - 4*c^2*e*f - 3*c*d*e*f + 4*e^2*f^2)

In [7]:
line = Line([y, z])

In [8]:
general_cubic = Cubic(cubic_new, line, sing_cubics)

In [9]:
SE10 = general_cubic.subs({b:-(c*c+e*f)/c, d:(3*e*f - c^2 + c*f + c*e)/(2*c)}).subs({f:c*(3*c-e)/(c+e)}).subs({e : (2-alpha)*c})

In [10]:
%time adm_SE10 = SE10.find_admissible_projectivities()

CPU times: user 840 ms, sys: 311 ms, total: 1.15 s
Wall time: 3.1 s


In [11]:
len(adm_SE10)

120

In [12]:
%time simm_SE10 = SE10.find_simmetries(adm_SE10)

CPU times: user 439 ms, sys: 69.5 ms, total: 509 ms
Wall time: 645 ms


In [13]:
len(simm_SE10)

120

In [14]:
perms = []
for el in simm_SE10:
    perms.append(Permutation(apply_proj_to_eck(el, SE10.eckardt_points)).to_cycles())

In [15]:
gp = PermutationGroup(perms)

In [16]:
gp.order()

120

In [17]:
gp.structure_description()

'S5'

In [18]:
nperms = [[cycle for cycle in perm if len(cycle)>1] for perm in perms]

In [19]:
gp.gens_small()

[(1,4,7)(3,5,9)(6,10,8), (1,10,8)(2,4,3,6,5,7)]

In [20]:
for gen in gp.gens_small():
    print(nperms.index(gen.cycle_tuples()))
    print(gen)

36
(1,7,10,3)(2,5,4,6)(8,9)
108
(1,8,2,9,7)(3,4,10,5,6)


In [23]:
gen1 = simm_SE10[36]
gen2 = simm_SE10[108]

In [24]:
gen1, gen2

(
[-3 -1  3  3]  [-1 -1 -3 -3]
[ 3  1 -1 -1]  [ 0  1  5  2]
[-1 -1  1 -1]  [ 0 -1 -1 -2]
[-1 -1 -1  1], [ 1  1 -1 -1]
)

In [25]:
Sylvester_planes_labels = [['E1', 'G3', 'F13'], ['E3', 'G4', 'F34'], ['E2', 'G1', 'F12'], ['E4', 'G2', 'F24'], ['F14', 'F23', 'F56']]

In [26]:
for label in Sylvester_planes_labels:
    print(label in SE10.eckardt_points_labels)

False
False
False
False
False


In [27]:
Sylvester_planes = [pl.plane/min(pl.plane.coefficients()) for pl in SE10.tritangent_planes if pl.labels in Sylvester_planes_labels]
Sylvester_planes

[-y + z, x + t, -x + y, x - 2*y + t, x - 1/2*y + 1/2*z]

In [28]:
plane_cubes = [pl^3 for pl in Sylvester_planes]
plane_cubes[2]*=8
plane_cubes[-1]*=8
plane_cubes[1]*=-1
plane_cubes[0]*=-1
plane_cubes

[y^3 - 3*y^2*z + 3*y*z^2 - z^3,
 -x^3 - 3*x^2*t - 3*x*t^2 - t^3,
 -8*x^3 + 24*x^2*y - 24*x*y^2 + 8*y^3,
 x^3 - 6*x^2*y + 12*x*y^2 - 8*y^3 + 3*x^2*t - 12*x*y*t + 12*y^2*t + 3*x*t^2 - 6*y*t^2 + t^3,
 8*x^3 - 12*x^2*y + 6*x*y^2 - y^3 + 12*x^2*z - 12*x*y*z + 3*y^2*z + 6*x*z^2 - 3*y*z^2 + z^3]

In [29]:
mon = ((x+y+z+t)^3).monomials()

In [30]:
nc = sum(plane_cubes)
[el for el in matrix([[nc.coefficient(mn) for mn in mon], [SE10.eqn.coefficient(mn) for mn in mon]]).minors(2) if el!=0]

[]

In [31]:
sost1 = change_coord(gen1)
sost2 = change_coord(gen2)

In [32]:
Sylvester_planes

[-y + z, x + t, -x + y, x - 2*y + t, x - 1/2*y + 1/2*z]

In [33]:
[True in [pl.subs(sost1).divides(pl1) for pl in Sylvester_planes] for pl1 in Sylvester_planes]

[True, True, True, True, True]

In [34]:
[True in [pl.subs(sost2).divides(pl1) for pl in Sylvester_planes] for pl1 in Sylvester_planes]

[True, True, True, True, True]